<a href="https://colab.research.google.com/github/Deok97/AI_personal_study/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4AI%EC%9D%91%EC%9A%A9_7%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 풀링연산 구현

In [1]:
import numpy as np

In [2]:
A = np.random.random((4, 4))
print(A)
stride = 1
filters = 2
maxPooling_result = np.zeros((A.shape[0]-stride, A.shape[1]-stride))

for row in range(0, A.shape[0]-1, stride):
  for col in range(0, A.shape[1]-1, stride):
    patch = A[row:row+filters, col:col+filters]
    maxPooling_result[row//stride, col//stride] = np.max(patch) # // 연산자가 나머지 버리고 정수 몫만을 구하는 연산자

print('filters = {}, strides = {}'.format(filters, stride))
print('Max Pooling Result: \n', maxPooling_result)

[[0.83009781 0.50154395 0.98974923 0.6410164 ]
 [0.46799584 0.3089244  0.61029286 0.75460007]
 [0.62810822 0.77118034 0.5508252  0.18966943]
 [0.51884114 0.48097654 0.97334334 0.03247394]]
filters = 2, strides = 1
Max Pooling Result: 
 [[0.83009781 0.98974923 0.98974923]
 [0.77118034 0.77118034 0.75460007]
 [0.77118034 0.97334334 0.97334334]]


### Functionalize Max Pooling operation

* strides = 1, 2 가능
* filters = 2만 적용 가능


In [ ]:
def maxPooling(value_matrix, filters, strides):
  stride = strides
  filters = filters
  maxPooling_result = np.zeros((A.shape[0]-stride, A.shape[1]-stride))

  for row in range(0, A.shape[0]-1, stride):
    for col in range(0, A.shape[1]-1, stride):
      patch = A[row:row+filters, col:col+filters]
      maxPooling_result[row//stride, col//stride] = np.max(patch)
  return maxPooling_result

In [ ]:
maxPooling(A, 1, 2)

array([[0.70101392, 0.72895984],
       [0.86822651, 0.45395607]])

# 맥스 풀링을 프레임워크 이용

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model
input = Input(shape=(4,4,1))
max_0 = MaxPooling2D(pool_size = (2,2), strides = 2)(input)
model = Model(input, max_0)

values = np.random.rand(4, 4)
# reshape 과정이 필요하다. (N, H, W, C)
resh_values = values.reshape(1, 4, 4, 1)
model.predict(resh_values)

array([[[[0.9328887 ],
         [0.9796505 ]],

        [[0.8392268 ],
         [0.95689034]]]], dtype=float32)

# 3차원 데이터 맥스 풀링 연산

In [ ]:
in0 = np.random.rand(4, 4)
in1 = np.random.rand(4, 4)
in2 = np.random.rand(4, 4)

# axis=-1은 마지막 차원으로 stack하는 것
values = np.stack([in0, in1, in2], axis = -1)

maxPooling_result = np.zeros([values.shape[0]-stride, values.shape[1]-stride, values.shape[2]])

for c in range(0, values.shape[2]):
  for row in range(0, values.shape[0]-1, stride):
      for col in range(0, values.shape[1]-1, stride):
        patch = values[row:row+filters, col:col+filters, c]
        maxPooling_result[row//stride, col//stride, c] = np.max(patch)
print(maxPooling_result)

[[[0.90974856 0.66121554 0.94817445]
  [0.74330376 0.46662227 0.896367  ]
  [0.71276065 0.74680429 0.99118723]]

 [[0.83035163 0.46662227 0.94817445]
  [0.74330376 0.46662227 0.896367  ]
  [0.71276065 0.74680429 0.50926914]]

 [[0.80364725 0.95212811 0.65843752]
  [0.64312407 0.95212811 0.96598771]
  [0.53104576 0.67916281 0.96598771]]]


# 3차원 맥스 풀링을 프레임워크를 이용

In [ ]:
resh_values = values.reshape(1, 4, 4, 3)
input = Input(shape=(4, 4, 3))
max_0 = MaxPooling2D(pool_size=(2,2), strides=2)(input)
model = Model(input, max_0)

model.summary()

result = model.predict(resh_values)
print(result)
print(result.shape)

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 4, 4, 3)]         0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 2, 2, 3)          0         
 g2D)                                                            
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
[[[[0.90974855 0.66121554 0.9481745 ]
   [0.7127606  0.7468043  0.9911872 ]]

  [[0.8036472  0.9521281  0.6584375 ]
   [0.53104573 0.6791628  0.9659877 ]]]]
(1, 2, 2, 3)


# 간단한 CNN 만들기

In [7]:
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist

# 데이터 불러오기 & 전처리
(train_images, train_labels), (test_images, test_labels) = \
mnist.load_data()
train_images = train_images/ 255.
test_images = test_images/ 255.

# 입력층
inputs = Input(shape=(32, 32))
res_inputs = Reshape((32, 32, 1))(inputs)

# layer 1
conv_0 = Conv2D(filters=6, kernel_size=5, strides=1, padding='same', activation='relu')(res_inputs)
max_0 = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(conv_0)

# layer 2
conv_1 = Conv2D(16, 5, 1, padding='same', activation='relu')(max_0)
max_1 = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(conv_1)

# layer 3
conv_2 = Conv2D(120, 5, 1, padding='same', activation='relu')(max_1)
flat_0 = Flatten()(conv_2)

# FC layer
# 매개변수의 이름은 'units'이다.
dense_0 = Dense(units=84, activation='relu')(flat_0)
outputs = Dense(10, activation='softmax')(dense_0)

# Model
model = Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32)]          0         
                                                                 
 reshape_4 (Reshape)         (None, 32, 32, 1)         0         
                                                                 
 conv2d_12 (Conv2D)          (None, 32, 32, 6)         156       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 16, 16, 6)        0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 16, 16, 16)        2416      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 8, 8, 16)         0         
 2D)                                                         